In [1]:
%load_ext jupyternotify

import numpy as np
import matplotlib
#allow for latex markup in matplotlib figures
matplotlib.rcParams['text.usetex'] = False
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap


#Import a few utility functions...
import sys  
from pathlib import Path
sys.path.insert(0, "../comm_channel")
sys.path.insert(0, "../polling_systems")
sys.path.insert(0, "../geometry")
sys.path.insert(0, "../utils")

#So we can import my local libs
import CommChannel as CC
import qos
import pointcloud as PC
import TSPN
import PollingSystem as PS
import MarkovianRP as MRP
import dtr
import motion
import shot_solvers as SHOT

import sim_template as sim
from importlib import reload 

<IPython.core.display.Javascript object>

In [2]:
#setup quality of service requirements using reasonable values
F5GHz, F2_4GHz = 5, 2.4 #typical wifi frequencies are 2.4, 5, and sometiems 6 GHz
MOBILE_TX_POWER = qos.Pwr(20)#100-200mW (20-23 dBm) pretty typical, with 1W (30dBm) the absolute maximum
print(MOBILE_TX_POWER)
r = 8#any of 2, 4, 6, or 8
rx_noise = qos.Pwr(-85)#based on Yan and MostofiTWC13 #1e-10#
print(rx_noise)
BER = 1e-6
GAMMA_TH = qos.QoSReq(BER, r, rx_noise).thresholdChannelGain(MOBILE_TX_POWER)
print(GAMMA_TH)
print('SNR Threshold: %d'%(GAMMA_TH+MOBILE_TX_POWER.dBm - rx_noise.dBm))

20.000000 dBm	-10.000000 dBW	100.000000 mW	0.100000 W
-85.000000 dBm	-115.000000 dBW	0.000000 mW	0.000000 W
-71.8297512825111
SNR Threshold: 33


In [3]:
#Setup the communication environment:

#For now, use same channel params
nPL = 5.2
kPL = -20

decorrSH = 2.09  
alpha = 16
sigmaSH = np.sqrt(alpha)
psdAtFC = 30

lam = 0.125
kRic = 10
decorrMP = 0.4*lam         
corrMP = 1
#sigmaMP = 3
sigmaMP = 1.5


#12 remote stations
l = 11
qBase = np.array([[2,6],[21,4],
                 [16,11],[28,23],
                 [10,22],[11,39],
                 [30, 11],[48, 8],
                 [18, 36],[35, 41],
                 [32, 33],[48, 25]])

sub_regions = [[25,0,10,0],
               [30,10, 25, 9],
               [20,0, 40, 17],
               [50, 25, 15, 3],
               [38, 15, 45, 27],
               [50, 30, 40, 20]]


x_max = 50
x_min = 0
y_max = 50
y_min = 0
region = [x_max, x_min, y_max, y_min]


cps = [CC.ChannelParams(qBase[i], nPL, kPL, sigmaSH, decorrSH, decorrMP, lam,
                      kRic, corrMP, psdAtFC, sigmaMP) for i in range(len(qBase))]



#res = 2/decorrMP
res = 10
n = 6
p_th = 0.5

#Polling System Setup
bandwidth = 2#leave bandwidth in MHz*1000000#give ourselfes 2MHz of BW
beta = 1/(r*bandwidth)
print(beta)
els = [0.3/beta, 0.2/beta, 0.3/beta, 0.01/beta, 0.01/beta, 0.02/beta]
print(els)

0.0625
[4.8, 3.2, 4.8, 0.16, 0.16, 0.32]


In [4]:
reload(sim)
ccs, cfs, true_joint_con_fields, tjcps = sim.create_channels(cps, region, res, GAMMA_TH, sub_regions)

Generating shadowing...
Shadowing generation complete.
Generating MP...
MP generation complete.
Generating shadowing...
Shadowing generation complete.
Generating MP...
MP generation complete.
Generating shadowing...
Shadowing generation complete.
Generating MP...
MP generation complete.
Generating shadowing...
Shadowing generation complete.
Generating MP...
MP generation complete.
Generating shadowing...
Shadowing generation complete.
Generating MP...
MP generation complete.
Generating shadowing...
Shadowing generation complete.
Generating MP...
MP generation complete.
Generating shadowing...
Shadowing generation complete.
Generating MP...
MP generation complete.
Generating shadowing...
Shadowing generation complete.
Generating MP...
MP generation complete.
Generating shadowing...
Shadowing generation complete.
Generating MP...
MP generation complete.
Generating shadowing...
Shadowing generation complete.
Generating MP...
MP generation complete.
Generating shadowing...
Shadowing genera

In [ ]:
reload(sim)
pcs, pfs, pjcps, prob_pred_con = sim.predict_channels(res, ccs, true_joint_con_fields, GAMMA_TH, p_th=p_th)
trismall_pcs = pcs

print(prob_pred_con)

Drawing 250 samples from Channel 1
Completed PredictedChannel 1
Drawing 250 samples from Channel 2
Completed PredictedChannel 2
Drawing 320 samples from Channel 3
Completed PredictedChannel 3
Drawing 320 samples from Channel 4
Completed PredictedChannel 4
Drawing 460 samples from Channel 5
Completed PredictedChannel 5
Drawing 460 samples from Channel 6
Completed PredictedChannel 6
Drawing 300 samples from Channel 7


In [ ]:
reload(sim)
sim.plotCFwithOverlay(n, tjcps, pjcps, qBase, region)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
#setup the delay-tolerant relay system
reload(dtr)
dt_sys = dtr.DTR(pcs, region, els, beta, th=GAMMA_TH, p_th = p_th)

In [ ]:
%store dt_sys

In [ ]:
#Calculate baseline TSPN policy
# import gurobi_solvers as GB
# reload(GB)
# reload(TSPN)
regions = dt_sys.cregions
tspn_min, tspn_argmin = TSPN.TSPN_BF(regions)
x=tspn_argmin[0]

reload(sim)
order = list(tspn_argmin[1])
order.append(0)#complete the loop
sim.plotCFwithOverlay(n, tjcps, pjcps, qBase, region)
plt.plot(x[order,0], x[order,1], zorder=100)
plt.scatter(x[:,0], x[:,1], zorder = 100)
plt.gca().invert_yaxis()
plt.show()

tspn_argmin[1]

In [ ]:
reload(dtr)
reload(sim)

vs=[0.2, 1, 5]
AORPS=[]
for v in vs:
    AORPS.append(sim.calc_AORP(dt_sys, v))

In [ ]:
# print(AORPs)
reload(sim)
sim.plot_AORP(dt_sys, AORPS[1], tjcps, pjcps, qBase, region)

In [ ]:
# reload(SHOT)
# pi = AORP['pi']
# x = AORP['X']
# S = np.zeros((n,n))
# for i in range(n):
#     for j in range(n):
#         S[i,j] = np.linalg.norm(x[i] - x[j])

# print(pi)
# print(S)
# P, val = SHOT.min_PWD_over_P(pi, S, verbose = False)


In [ ]:
%%notify
reload(sim)
TSPNP={'X':x, 'SEQ':tspn_argmin[1]}
hrs = 1
mins = 0
seconds = 0
for i in range(len(vs)):
    v = vs[i]
    print('Velocity = %f'%(v))
    AORP = AORPS[i]
    motion_power = motion.LinearMotionPower(7.2, 0.29, v)
    sim.run_sims(dt_sys.ps, AORP, TSPNP, hrs, mins, seconds, motion_power, MOBILE_TX_POWER.W, v)


In [ ]:
# seconds = 60**2
# pi = AORP['pi']
# S =  dtr.XtoS(AORP['X'],vs[-1])
# for a in [0.0001, 0.1, 0.2, 0.5, 0.8, 0.9,0.9999]:    
#     mrp = MRP.rnd2P(pi, S, alpha=a)
#     MRP_res, MRP_xt = sim.runsimsforpolicy(ps, mrp, S, motion_power, MOBILE_TX_POWER.W, seconds)
#     print('MRP\t---\t%.2f\t%.2f\t%.2f\t%.2f'%(MRP_res['WT'], MRP_res['E'], MRP_res['MBS'], MRP_res['MBR'] ))

In [ ]:
# import StaticRP as SRP
# #0.01 is too precise
# #0.05 seems "close enough"
# for e in [0.1, 0.05]:
#     rtable = SRP.SRPFromPis(pi, eps=e)
#     rtable_res, rtable_xt = sim.runsimsforpolicy(ps, rtable, S, motion_power, MOBILE_TX_POWER.W, seconds)
#     print('Tab\t---\t%.2f\t%.2f\t%.2f\t%.2f'%(rtable_res['WT'], rtable_res['E'], rtable_res['MBS'], rtable_res['MBR']))

In [ ]:
reload(sim)
sim.plot_TSPNP(dt_sys, TSPNP, tjcps, pjcps, qBase, region)
sim.save_plt('six_TSPNP.png')

In [ ]:
reload(sim)
sim.plot_AORP(dt_sys, AORPS[0], tjcps, pjcps, qBase, region)
sim.save_plt('six_AORP_v02.png')

In [ ]:
reload(sim)
sim.plot_AORP(dt_sys, AORPS[1], tjcps, pjcps, qBase, region)
sim.save_plt('six_AORP_v1.png')

In [ ]:
reload(sim)
sim.plot_AORP(dt_sys, AORPS[2], tjcps, pjcps, qBase, region)
sim.save_plt('six_AORP_v5.png')

In [ ]:
%%notify
#####################
# Now for the same system, re-run, but for different system lambdas
#els2 = [0.15/beta, 0.15/beta, 0.15/beta, 0.15/beta, 0.15/beta, 0.15/beta]
els2 = [0.01/beta, 0.01/beta, 0.02/beta, 0.3/beta, 0.2/beta, 0.3/beta]
dt_sys.ps = PS.PollingSystem(els2, beta)

AORPS2=[]
for v in vs:
    AORPS2.append(sim.calc_AORP(dt_sys, v))
    print('Velocity = %f'%(v))
    AORP = AORPS2[-1]
    motion_power = motion.LinearMotionPower(7.2, 0.29, v)
    sim.run_sims(dt_sys.ps, AORP, TSPNP, hrs, mins, seconds, motion_power, MOBILE_TX_POWER.W, v)


In [ ]:
reload(sim)
sim.plot_AORP(dt_sys, AORPS2[0], tjcps, pjcps, qBase, region)
sim.save_plt('six_AORP_v02_L2.png')

In [ ]:
reload(sim)
sim.plot_AORP(dt_sys, AORPS2[1], tjcps, pjcps, qBase, region)
sim.save_plt('six_AORP_v1_L2.png')

In [ ]:
reload(sim)
sim.plot_AORP(dt_sys, AORPS2[2], tjcps, pjcps, qBase, region)
sim.save_plt('six_AORP_v5_L2.png')

In [ ]:
%%notify
#####################
# Now for the same system, re-run, but for different system lambdas
els3 = [0.15/beta, 0.15/beta, 0.15/beta, 0.15/beta, 0.15/beta, 0.15/beta]
dt_sys.ps = PS.PollingSystem(els3, beta)

AORPS3=[]
for v in vs:
    AORPS3.append(sim.calc_AORP(dt_sys, v))
    print('Velocity = %f'%(v))
    AORP = AORPS3[-1]
    motion_power = motion.LinearMotionPower(7.2, 0.29, v)
    sim.run_sims(dt_sys.ps, AORP, TSPNP, hrs, mins, seconds, motion_power, MOBILE_TX_POWER.W, v)


In [ ]:
reload(sim)
sim.plot_AORP(dt_sys, AORPS2[0], tjcps, pjcps, qBase, region)
sim.save_plt('six_AORP_v02_L3.png')

In [ ]:
reload(sim)
sim.plot_AORP(dt_sys, AORPS2[1], tjcps, pjcps, qBase, region)
sim.save_plt('six_AORP_v1_L2.png')

In [ ]:
reload(sim)
sim.plot_AORP(dt_sys, AORPS2[2], tjcps, pjcps, qBase, region)
sim.save_plt('six_AORP_v5_L2.png')